# Solve a generalized assignment problem using Lagrangian relaxation

This tutorial includes data and information that you need to set up decision optimization engines and build mathematical programming models to solve a Generalized Assignment Problem using Lagrangian relaxation.

Some familiarity with Python is recommended. This notebook runs on the latest version of Python.

## Table of contents
* [Describe the business problem](#Describe-the-business-problem)
* [How Decision Optimization can help](#How-decision-optimization-can-help) 
* [Use Decision Optimization to create and solve the model](#Use-Decision-Optimization)
* [Summary](#Summary)<br> 

## Describe the business problem   


This notebook illustrates how to solve an optimization model using Lagrangian relaxation techniques. 
It solves a generalized assignment problem (GAP), as defined by Wolsey, using this relaxation technique.

The main aim is to show multiple optimization through the modification of different models existing in a single environment, and not to show you how to solve a GAP problem.

In the field of Mathematical Programming, this technique consists of approximating a difficult constrained problem by a simpler problem: you remove difficult constraints by integrating them in the objective function, and penalizing it if the constraint is not respected.

The method penalizes violations of inequality constraints using a Lagrange multiplier, which imposes a cost on violations. These added costs are used instead of the strict inequality constraints in the optimization. In practice, this relaxed problem can often be solved more easily than the original problem.

For more information, see the following Wikipedia articles: <a href="https://en.wikipedia.org/wiki/Generalized_assignment_problem" target="_blank" rel="noopener noreferrer">Generalized assignment problem</a> and <a href="https://en.wikipedia.org/wiki/Lagrangian_relaxation" target="_blank" rel="noopener noreferrer">Lagrangian relaxation</a>.

This notebook first solves the standard problem (which is not important here), then shows you how to reformulate it to meet the Lagrangian Relaxation features.

## How decision optimization can help

Prescriptive analytics (decision optimization) technology recommends actions that are based on desired outcomes. It considers specific scenarios, resources, and knowledge of past and current events. With this insight, your organization can make better decisions and have greater control over business outcomes.

Prescriptive analytics is the next step on the path to insight-based actions. It creates value through synergy with predictive analytics, which analyzes data to predict future outcomes. Prescriptive analytics takes that insight to the next level by suggesting the optimal way to handle a future situation. Organizations that act fast in dynamic conditions and make superior decisions in uncertain environments gain a strong competitive advantage.

With prescriptive analytics, you can:

* Automate the complex decisions and trade-offs to better manage your limited resources.
    
* Take advantage of a future opportunity or mitigate a future risk.
    
* Proactively update recommendations based on changing events.
    
* Meet operational goals, increase customer loyalty, prevent threats and fraud, and optimize business processes.





## Use Decision Optimization
Perform the following steps to create and solve the model.

1. [Model the Data](#1.-Model-the-data)<br>
2. [Set up the prescriptive model](#2.-Set-up-the-prescriptive-model)<br>
      2.1 [Create the DOcplex model](#2.1-Create-the-DOcplex-model)<br>
      2.2 [Define the decision variables](#2.2-Define-the-decision-variables)<br>
      2.3 [Define the business constraints](#2.3-Define-the-business-constraints)<br>
      2.4 [Solve the model](#2.4-Solve-the-model)<br>
      2.5 [Solve the model with Lagrangian Relaxation](#2.5-Solve-the-model-with-Lagrangian-Relaxation-method)<br>
3. [Investigate the solution and run an example analysis](#3.-Investigate-the-solution-and-run-an-example-analysis)<br>

### 1. Model the data
In this scenario, the data is simple. It is delivered as 3 input arrays: A, B, and C. The data does not need changing or refactoring.

In [1]:
B = [15, 15, 15]
C = [
    [ 6, 10, 1],
    [12, 12, 5],
    [15,  4, 3],
    [10,  3, 9],
    [8,   9, 5]
]
A = [
    [ 5,  7,  2],
    [14,  8,  7],
    [10,  6, 12],
    [ 8,  4, 15],
    [ 6, 12,  5]
]

### 2. Set up the prescriptive model

Start by viewing the environment information. This information should be updated when you run the notebook.
 

In [ ]:
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

You will first create an optimization problem, composed of 2 basic constraint blocks, then you will resolve it using Lagrangian Relaxation on one of the constraint blocks.

#### 2.1 Create the DOcplex model
The model contains the business constraints and the objective.


In [3]:
from docplex.mp.model import Model

mdl = Model("GAP per Wolsey")

#### 2.2 Define the decision variables

In [4]:
print("#As={}, #Bs={}, #Cs={}".format(len(A), len(B), len(C)))
number_of_cs = len(C)
# variables
x_vars = [mdl.binary_var_list(c, name=None) for c in C]

#As=5, #Bs=3, #Cs=5


#### 2.3 Define the business constraints

In [5]:
# constraints
cts = mdl.add_constraints(mdl.sum(xv) <= 1 for xv in x_vars)

mdl.add_constraints(mdl.sum(x_vars[ii][j] * A[ii][j] for ii in range(number_of_cs)) <= bs for j, bs in enumerate(B))

# objective
total_profit = mdl.sum(mdl.scal_prod(x_i, c_i) for c_i, x_i in zip(C, x_vars))
mdl.maximize(total_profit)
mdl.print_information()

Model: GAP per Wolsey
 - number of variables: 15
   - binary=15, integer=0, continuous=0
 - number of constraints: 8
   - linear=8
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP


#### 2.4 Solve the model 

Use the Decision Optimization to solve the model. 

In [6]:
s = mdl.solve()
assert s is not None
obj = s.objective_value
print("* GAP with no relaxation run OK, best objective is: {:g}".format(obj))

* GAP with no relaxation run OK, best objective is: 46


#### 2.5 Solve the model with Lagrangian Relaxation method

Now consider, for the purpose of demonstration of the Lagrangian Relaxation, that this model was hard to solve for CPLEX.
You will approximate this problem by using an iterative model, where the objective is modified at each iteration. 

(Wait a few seconds for the solution, due to a time limit parameter.)

You first remove the offending constraints from the model

In [7]:
for ct in cts:
    mdl.remove_constraint(ct)

In [8]:
#p_vars are the penalties attached to violating the constraints
p_vars = mdl.continuous_var_list(C, name='p')  # new for relaxation

In [9]:
# new version of the approximated constraint where we apply the penalties
mdl.add_constraints(mdl.sum(xv) == 1 - pv for xv, pv in zip(x_vars, p_vars))
;

''

In [10]:
#Define the maximum number of iterations
max_iters = 10

In [11]:
number_of_cs = len(C)
c_range = range(number_of_cs)

In [12]:
# Langrangian relaxation loop 
eps = 1e-6
loop_count = 0
best = 0
initial_multiplier = 1
multipliers = [initial_multiplier] * len(C)

# Objective function
# I'd write the key perfromance indicator (kpi) as
# total_profit = mdl.sum(mdl.sum(x_vars[task][worker] * C[task][worker]) for task, worker in zip(tasks, workers))
total_profit = mdl.sum(mdl.scal_prod(x_i, c_i) for c_i, x_i in zip(C, x_vars))
mdl.add_kpi(total_profit, "Total profit")
print("starting the loop")

starting the loop


In [13]:
while loop_count <= max_iters:
    loop_count += 1
    # Rebuilt at each loop iteration
    total_penalty = mdl.scal_prod(p_vars, multipliers)
    
    mdl.maximize(total_profit + total_penalty)
    s = mdl.solve()
    if not s:
        print("*** solve fails, stopping at iteration: %d" % loop_count)
        break
    best = s.objective_value
    penalties = [pv.solution_value for pv in p_vars]
    print('%d> new lagrangian iteration:\n\t obj=%g, m=%s, p=%s' % (loop_count, best, str(multipliers), str(penalties)))

    do_stop = True
    justifier = 0
    for k in c_range:
        penalized_violation = penalties[k] * multipliers[k]
        if penalized_violation >= eps:
            do_stop = False
            justifier = penalized_violation
            break

    if do_stop:
        print("* Lagrangian relaxation succeeds, best={:g}, penalty={:g}, #iterations={}"
                .format(best, total_penalty.solution_value, loop_count))
        break
    else:
        # Update multipliers and start the loop again.
        scale_factor = 1.0 / float(loop_count)
        multipliers = [max(multipliers[i] - scale_factor * penalties[i], 0.) for i in c_range]
        print('{0}> -- loop continues, m={1!s}, justifier={2:g}'.format(loop_count, multipliers, justifier))

1> new lagrangian iteration:
	 obj=47, m=[1, 1, 1, 1, 1], p=[0, 0, 0, 0, 1.0]
1> -- loop continues, m=[1.0, 1.0, 1.0, 1.0, 0.0], justifier=1
2> new lagrangian iteration:
	 obj=46, m=[1.0, 1.0, 1.0, 1.0, 0.0], p=[0, 0, 0, 0, 1.0]
* Lagrangian relaxation succeeds, best=46, penalty=0, #iterations=2


In [14]:
print(best)

46.0


### 3. Investigate the solution and run an example analysis

You can see that with this relaxation method applied to this simple model, we find the same solution to the problem.

## Summary


You have learned how to set up and use IBM Decision Optimization CPLEX Modeling for Python to formulate a Constraint Programming model and solve it with IBM Decision Optimization.

## References
* <a href="https://rawgit.com/IBMDecisionOptimization/docplex-doc/master/docs/index.html" target="_blank" rel="noopener noreferrer">Decision Optimization CPLEX Modeling for Python documentation</a>
* <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/welcome-main.html" target="_blank" rel="noopener noreferrer">Online documentation</a>
* Need help with DOcplex or to report a bug? Go to <a href="https://developer.ibm.com/answers/smartspace/docloud" target="_blank" rel="noopener noreferrer">https://developer.ibm.com/answers/smartspace/docloud</a>


<hr>
Copyright © 2017-2021. This notebook and its source code are released under the terms of the MIT License.